# Import Libraries

In [2]:
import os
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau

import cv2
import mediapipe as mp

# Define Dataset Directories

In [3]:
TRAIN_DIR = "./output/train/"
TEST_DIR = "./output/test/"
IMG_SIZE = 48

# Image Augmentation

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=3,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 128

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

test_gen = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

Found 28869 images belonging to 24 classes.
Found 1967 images belonging to 24 classes.


# Get available labels/alphabets

In [5]:
labels = list(train_gen.class_indices.keys())
print(labels)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']


# Create CNN Model & Layers

In [5]:
model2 = Sequential()
model2.add(Conv2D(128, kernel_size=(5,5),
                 strides=1, padding='same', activation='relu', input_shape=(48,48,1)))
model2.add(MaxPool2D(pool_size=(4,4), strides=2, padding='same'))
model2.add(Dropout(0.4))

model2.add(Conv2D(64, kernel_size=(2,2),
                 strides=1, activation='relu', padding='same'))
model2.add(MaxPool2D((4,4), 2, padding='same'))
model2.add(Dropout(0.4))

model2.add(Conv2D(32, kernel_size=(2,2),
                 strides=1, activation='relu', padding='same'))
model2.add(MaxPool2D((4,4), 2, padding='same'))
model2.add(Dropout(0.4))
          
model2.add(Flatten())
model2.add(Dense(units=512,activation='relu'))
model2.add(Dropout(rate=0.25))
model2.add(Dense(units=24,activation='softmax'))
model2.summary()

model2.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 128)       3328      
                                                                 
 max_pooling2d (MaxPooling2  (None, 24, 24, 128)       0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 24, 24, 128)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        32832     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 12, 12, 64)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 12, 12, 64)        0

# Model Fit

In [80]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)

model2.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // batch_size,
    epochs=20,
    validation_data=test_gen,
    validation_steps=test_gen.samples // batch_size,
    callbacks = [learning_rate_reduction]
)

Epoch 1/20
225/225 [==============================] - 274s 1s/step - loss: 1.9754 - accuracy: 0.3655 - val_loss: 0.8952 - val_accuracy: 0.7510 - lr: 0.0010
Epoch 2/20
225/225 [==============================] - 142s 632ms/step - loss: 0.7305 - accuracy: 0.7333 - val_loss: 0.6914 - val_accuracy: 0.7937 - lr: 0.0010
Epoch 3/20
225/225 [==============================] - 149s 663ms/step - loss: 0.4406 - accuracy: 0.8420 - val_loss: 0.7170 - val_accuracy: 0.7906 - lr: 0.0010
Epoch 4/20
225/225 [==============================] - 147s 651ms/step - loss: 0.3152 - accuracy: 0.8880 - val_loss: 0.6842 - val_accuracy: 0.8177 - lr: 0.0010
Epoch 5/20
225/225 [==============================] - 154s 682ms/step - loss: 0.2255 - accuracy: 0.9203 - val_loss: 0.6232 - val_accuracy: 0.7969 - lr: 0.0010
Epoch 6/20
225/225 [==============================] - ETA: 0s - loss: 0.1934 - accuracy: 0.9320
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
225/225 [===========================

In [81]:
model2.save("./output/model_all_alpha_85.h5")

In [3]:
model2 = load_model("./models/model_all_alpha_85.h5")

In [4]:
model2.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_56 (Conv2D)          (None, 48, 48, 128)       3328      
                                                                 
 max_pooling2d_55 (MaxPooli  (None, 24, 24, 128)       0         
 ng2D)                                                           
                                                                 
 dropout_81 (Dropout)        (None, 24, 24, 128)       0         
                                                                 
 conv2d_57 (Conv2D)          (None, 24, 24, 64)        32832     
                                                                 
 max_pooling2d_56 (MaxPooli  (None, 12, 12, 64)        0         
 ng2D)                                                           
                                                                 
 dropout_82 (Dropout)        (None, 12, 12, 64)      

# Second Round Training Attempt

In [27]:
model2 = load_model("./models/model_all_alpha_85.h5")

# Freeze all layers in the model
for layer in model2.layers:
    layer.trainable = False

# Recompile
model2.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)

model2.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // batch_size,
    epochs=20,
    validation_data=test_gen,
    validation_steps=test_gen.samples // batch_size,
    callbacks = [learning_rate_reduction]
)

Epoch 1/20
17/17 [==============================] - 14s 780ms/step - loss: 8.8690 - accuracy: 0.2693 - val_loss: 0.6058 - val_accuracy: 0.8568 - lr: 0.0010
Epoch 2/20
17/17 [==============================] - 11s 661ms/step - loss: 8.8694 - accuracy: 0.2488 - val_loss: 0.5999 - val_accuracy: 0.8568 - lr: 0.0010
Epoch 3/20
17/17 [==============================] - 11s 664ms/step - loss: 8.8936 - accuracy: 0.2697 - val_loss: 0.6042 - val_accuracy: 0.8573 - lr: 0.0010
Epoch 4/20
17/17 [==============================] - 11s 677ms/step - loss: 9.0334 - accuracy: 0.2633 - val_loss: 0.5950 - val_accuracy: 0.8583 - lr: 0.0010
Epoch 5/20
17/17 [==============================] - 11s 663ms/step - loss: 8.9902 - accuracy: 0.2564 - val_loss: 0.6097 - val_accuracy: 0.8562 - lr: 0.0010
Epoch 6/20
17/17 [==============================] - ETA: 0s - loss: 8.8401 - accuracy: 0.2774
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
17/17 [==============================] - 12s 702ms

In [28]:
model2.save("./models/test.h5")